In [12]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U einops
# !pip install -q -U safetensors
# !pip install -q -U torch
# !pip install -q -U xformers
# !pip install -q -U langchain
# !pip install -q -U ctransformers[cuda]
# !pip install chromadb
# !pip install sentence-transformers

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from IPython.display import Audio,display

#!huggingface-cli login

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

audio=display(display(Audio('heard.mp3',autoplay=True)))

text=pipe('heard.mp3')

new_text=''
for i in range(len(text['chunks'])):
    new_text+=text['chunks'][i]['text']+'\n'

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, resume_download=True)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

prompt = "Please summarize the main points discussed in the conversation:"
eval_prompt = f"{prompt} {new_text}"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


None

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Please summarize the main points discussed in the conversation:  Who is that? That man?
 Who is that? Who is...
 That is Sirius Black, that is.
 Don't tell me you've never been hearing of Sirius Black.
 He's a murderer.
 Got himself locked up in Azkaban for it.
 How did he escape?
 Well, that's the question, isn't it?
 He's the first one that done it.
 He was a big supporter of...
 You-know-who.
 Reckon you've heard of him?
 Yeah.
 Him I've heard of.
 But not this bloke.
 What do you know about him?
 Nothing.
 Just what they say on the news.
 They say he killed thirteen people.
 Thirteen Muggles.
 And his own parents.
 His father was a wizard.
 His mother wasn't.
 She married him anyway.
 Then she found out he was a wizard and left him.
 So he went after her.
 Killed her and their son.
 Then he went after his dad.
 Killed him too.
 Then he went after his aunt and uncle.
 Killed them as well.
 Then he went after his cousin.
 Dudley Dursley.
 He didn't kill him.
 No.
 He just tried to ea